In [39]:
#Pacotes que estamos importando
import numpy as np               
import pandas as pd             
from scipy.stats import rankdata # Para o ranking das candidatas

In [55]:
# Dados

attributes = np.array(["Beneficio 1", "Beneficio 1", "Custo 1", "Custo 2"])
candidates = np.array(["A1", "A2", "A3", "A4","A5"])
raw_data = np.array([
    [450, 8000,  54,  145],
    [10, 9100,  2,  160],
    [100, 8200,  31,   153],
    [220, 9300,  1,  162],
    [5, 8400,  23,   158],
    
])

# Indices das alternativas de lucro (contagem iniciando do zero).
# Indices não listados são considerados de custo.
benefit_attributes = set([0,1])


#MEREC


#Step 1 - Do MEREC
#Normalização

m = len(raw_data)
n = len(attributes)

#Criando um vetor de tamanho n fazio e logo em seguida estou preenchendo cada vetor com o tamanho máximo da coluna 
divisors_merec = np.empty(n)
for j in range(n):
    column = raw_data[:,j]
    divisors_merec[j] = np.max(column)


divisors_merec = np.empty(n)
for j in range(n):
    column = raw_data[:, j]
    if j in benefit_attributes:
        divisors_merec[j] = np.min(column)
    else:
        divisors_merec[j] = np.max(column)

        
for j in range(n):
    column = raw_data[:, j]
    if j in benefit_attributes:
        normalized_data[:, j] = divisors_merec[j] / column
    else:
        normalized_data[:, j] = column / divisors_merec[j]

pd.DataFrame(data=normalized_data, index=candidates, columns=attributes)







,Beneficio 1,Beneficio 1,Custo 1,Custo 2
A1,0.011111,1.000000,1.000000,0.895062
A2,0.500000,0.879121,0.037037,0.987654
A3,0.050000,0.975610,0.574074,0.944444
A4,0.022727,0.860215,0.018519,1.000000
A5,1.000000,0.952381,0.425926,0.975309


In [41]:
#Step 3 - Do MEREC

#Calculo do Overall de Cada Alternativa

m = normalized_data.shape[1]  # número de colunas
n = normalized_data.shape[0]  # número de linhas

S_0 = np.empty(n)  # vetor para armazenar os valores de S_i

for i in range(n):
    row = normalized_data[i, :]
    S[i] = np.log(1 + (1 / m) * np.sum(np.abs(np.log(row))))

print(S_0)

[0.89506173 0.98765432 0.94444444 1.         0.97530864]


In [57]:
#Step 4 - DO MEREC

m, n = normalized_data.shape

S = np.zeros((m, n))

for i in range(m):
    for j in range(n):
        S[i, j] = np.log(1 + (1 / (n)) * np.sum(np.abs(np.log(normalized_data[i, :][j != np.arange(n)]))))

pd.DataFrame(data=S, index=candidates, columns=attributes)


,Beneficio 1,Beneficio 1,Custo 1,Custo 2
A1,0.027339,0.766708,0.766708,0.753749
A2,0.620186,0.693323,0.189463,0.707767
A3,0.147740,0.642891,0.570638,0.638615
A4,0.710441,1.079529,0.684959,1.092238
A5,0.208493,0.198542,0.018280,0.203406


In [43]:
#Step 5

# Calculando as diferenças entre as colunas de S e o vetor S_0
diff = np.abs(S - S_0.reshape(-1, 1))

# Somando as diferenças ao longo do eixo das linhas
E_j = np.sum(diff, axis=0)

print(E_j)

[3.0882702  1.58053448 2.57242093 1.59116968]


In [44]:
#Step 6

# Calculando a soma de todas as entradas de E_j
sum_E_j = np.sum(E_j)

# Dividindo cada entrada de E_j pela soma
weights = np.divide(E_j, sum_E_j)

print(weights)

[0.34965262 0.17894744 0.29124839 0.18015155]


In [51]:
#Aqui começará o WISP

#print(raw_data)



#Step 1
#Normalização

m = len(raw_data)
n = len(attributes)

#Criando um vetor de tamanho n fazio e logo em seguida estou preenchendo cada vetor com o tamanho máximo da coluna 
divisors = np.empty(n)
for j in range(n):
    column = raw_data[:,j]
    divisors[j] = np.max(column)

#Dividindo cada termo pelo divisor
#print(raw_data)
#print(divisors)
raw_data = raw_data.astype('float64')
raw_data /= divisors

#Matriz de Decisão
pd.DataFrame(data=raw_data, index=candidates, columns=attributes)


,Beneficio 1,Beneficio 1,Custo 1,Custo 2
A1,1.000000,0.860215,1.000000,0.895062
A2,0.022222,0.978495,0.037037,0.987654
A3,0.222222,0.881720,0.574074,0.944444
A4,0.488889,1.000000,0.018519,1.000000
A5,0.011111,0.903226,0.425926,0.975309


In [52]:
#Step 2

# Criando uma matriz com todas as colunas zerados com o comprimento dos critérios
all_columns = np.zeros((raw_data.shape[0], len(attributes)))

# Preenchendo as colunas de lucro com os valores correspondentes
all_columns[:, list(benefit_attributes)] = raw_data[:, list(benefit_attributes)]

#print(all_columns)

benefit_profit = all_columns*weights

#print(benefit_profit)
cost_profit = (raw_data - all_columns)*weights

#Pra não atrapalhar, vou tirar as colunas que não são zeros 

# Verificar se todos os elementos de cada coluna são iguais a zero
nonzero_columns = np.all(benefit_profit != 0, axis=0) 
benefit_profit = benefit_profit[:, nonzero_columns] # Remover as colunas cujos elementos são todos iguais a zero

#print(benefit_profit)

# Verificar se todos os elementos de cada coluna são iguais a zero
nonzero_columns = np.all(cost_profit != 0, axis=0) 
cost_profit = cost_profit[:, nonzero_columns] # Remover as colunas cujos elementos são todos iguais a zero

#print(cost_profit)

column_sums = np.sum(benefit_profit, axis=1)
column_sums2 =np.sum(cost_profit, axis=1)
column_products = np.prod(benefit_profit, axis=1)
column_products2 = np.prod(cost_profit, axis=1)
u_i_wsd = column_sums - column_sums2
u_i_wpd = column_products- column_products2

#Aqui vou fazer um if para caso só tenhamos critérios de lucro ou só critério de custo:

if np.any(cost_profit) and np.any(benefit_profit):# Ambas as matrizes são diferentes de zero
    u_i_wsr = column_sums/column_sums2
    u_i_wpr = column_products/column_products2
    
# A matriz cost_profit é diferente de zero e a matriz benefit_profit é igual a zero
#Ou seja só tem critério de custo
elif np.any(cost_profit) and not np.any(benefit_profit):
    u_i_wsr = 1/column_sums2
    u_i_wpr = 1/column_products2

#Aqui estamos com cost_profit igual a 0
else :
    
    u_i_wsr = column_sums
    u_i_wpr = column_products

#print(cost_profit)
#print(benefit_profit)
#print(row_sums)
#print(row_sums2)
#print(benefit_profit)
#print(cost_profit)
#print(column_sums)
#print(column_sums2)

#print(column_products)
#print(column_products2)
#print(column_products)
#print(column_products2)

#print(u_i_wsd)
#print(u_i_wpd)
#print(u_i_wsr)
#print(u_i_wpr)

df = pd.DataFrame({
    'u_i_wsd': u_i_wsd,
    'u_i_wpd': u_i_wpd,
    'u_i_wsr': u_i_wsr,
    'u_i_wpr': u_i_wpr
},index=candidates)

# Exibir o DataFrame
df

,u_i_wsd,u_i_wpd,u_i_wsr,u_i_wpr
A1,0.051091,0.006860,1.112909,1.146080
A2,-0.005845,-0.000559,0.969026,0.708868
A3,-0.101859,-0.016188,0.698053,0.430958
A4,0.164344,0.029618,1.885735,31.482170
A5,-0.134239,-0.021168,0.552170,0.028810


In [53]:
#Step 3

u_i_wsd2 = u_i_wsd/(1+np.max(u_i_wsd))
u_i_wpd2 = u_i_wpd/(1+np.max(u_i_wpd))
u_i_wsr2 = u_i_wsr/(1+np.max(u_i_wsr))
u_i_wpr2 = u_i_wpr/(1+np.max(u_i_wpr))

df2 = pd.DataFrame({
    'u_i_wsd2': u_i_wsd2,
    'u_i_wpd2': u_i_wpd2,
    'u_i_wsr2': u_i_wsr2,
    'u_i_wpr2': u_i_wpr2
}, index = candidates)


df2

,u_i_wsd2,u_i_wpd2,u_i_wsr2,u_i_wpr2
A1,0.043879,0.006663,0.385659,0.035283
A2,-0.005020,-0.000543,0.335799,0.021823
A3,-0.087482,-0.015722,0.241898,0.013268
A4,0.141147,0.028766,0.653468,0.969214
A5,-0.115291,-0.020559,0.191345,0.000887


In [54]:
column_sums = df2.sum(axis=1)/4

#Fazendo o ranquamento
ranked_column_sums = column_sums.rank(ascending=False).sort_values(ascending=True).reset_index()
ranked_column_sums = ranked_column_sums.rename(columns={'index': 'Alternativa', 0: 'Rank'})

# Exibir o resultado do ranqueamento formatado
print(ranked_column_sums)

  Alternativa  Rank
0          A4   1.0
1          A1   2.0
2          A2   3.0
3          A3   4.0
4          A5   5.0
